In [ ]:
from pytrends.request import TrendReq
import pandas as pd
from datetime import datetime, timedelta
import time
from tqdm import tqdm  # Import tqdm for the progress bar

# Initialize pytrends
pytrends = TrendReq(hl='en-US', tz=360)

# Define the start and end date for the complete duration
start_date = datetime(2014, 9, 17)
end_date = datetime(2024, 4, 17)

# Calculate the end date for each 3-month period
three_months = timedelta(days=90)

# Initialize an empty list to hold all the data
all_data = []

# Calculate the number of periods to iterate through
total_periods = (end_date - start_date) // three_months
total_periods = total_periods + 1 if (end_date - start_date) % three_months != timedelta(0) else total_periods

# Loop through the date range in steps of 3 months with tqdm for the progress bar
current_date = start_date
for _ in tqdm(range(total_periods)):  # tqdm is used here to show progress
    # Calculate the end of the three-month period or use the end date
    three_month_end = min(current_date + three_months, end_date)
    
    # Format the dates in the required string format
    start_str = current_date.strftime('%Y-%m-%d')
    end_str = three_month_end.strftime('%Y-%m-%d')
    
    # Build the payload for the request
    pytrends.build_payload(kw_list=['bitcoin'], timeframe=f'{start_str} {end_str}')
    
    # Get the interest over time
    data = pytrends.interest_over_time()
    
    # Drop the 'isPartial' column
    data = data.drop(columns='isPartial', errors='ignore')
    
    # Append the data to the list
    all_data.append(data)
    
    # Sleep for 60 seconds to avoid sending too many requests in a short time
    time.sleep(90)
    
    # Update the current date to the next period
    current_date = three_month_end

# Concatenate all dataframes in the list
all_data_df = pd.concat(all_data)

# Reset index of the final dataframe
all_data_df.reset_index(inplace=True)

# Show the DataFrame
print(all_data_df)

all_data_df.to_csv('GtrendsD_end.csv', index=False)


In [2]:
from pytrends.request import TrendReq
import pandas as pd
from datetime import datetime, timedelta
import time
from tqdm import tqdm  # Import tqdm for the progress bar

start_date = datetime(2014, 9, 17)
end_date = datetime(2024, 4, 17)

# Initialize pytrends
pytrends = TrendReq(hl='en-US', tz=360)
pytrends.build_payload(kw_list=['bitcoin'], timeframe=('2014-9-17 2024-4-17'))
    
    # Get the interest over time
data = pytrends.interest_over_time()


# Reset index of the final dataframe
data.rename_axis('Date').reset_index().to_csv('../Data/Gtrends_Monthly.csv', index=False)


In [6]:
df = pd.read_csv('../Data/GtrendsD_2014-2015-04-17.csv')
all_data_df = pd.read_csv('../Data/GtrendsD_end.csv')
merged = pd.concat([df, all_data_df], axis=0)

month = pd.read_csv('../Data/Gtrends_Monthly.csv')

# Create new column with the month and year
merged['Month'] = pd.to_datetime(merged['date']).dt.to_period('M')
merged['date'] = pd.to_datetime(merged['date'])

month['Date'] = pd.to_datetime(month['Date'])
month['Month'] = month['Date'].dt.to_period('M')

adjusted = pd.merge(merged, month, on='Month', how='left')

adjusted['Weighted_interest'] = adjusted['bitcoin_x'] * (adjusted['bitcoin_y']/100)

adjusted.drop(columns=['bitcoin_x', 'bitcoin_y', 'Date', 'Month', 'isPartial'], inplace=True)

adjusted.rename(columns={'date': 'Date', 'Weighted_interest': 'GTrends_Interest'}, inplace=True)

adjusted.to_csv('../Cleaned_Data/Adjusted_Gtrends.csv', index=False)